# Data preprocessing notebook

This is part of the repository to show where the data comes from and how it's transformed.

It's not meant to be nice and easy to read, it's basically some draft snippets.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from unidecode import unidecode
import requests
import json
import matplotlib.pyplot as plt

In [2]:
# https://www.insee.fr/fr/statistiques/8595130?sommaire=8595113
df_all = pd.read_parquet("prenoms-2024.parquet").sort_values("periode")
df_all["periode"] = df_all["periode"].map(lambda x: int(x))
df_all["valeur"] = df_all["valeur"].map(lambda x: int(x))

df_all = df_all[df_all["periode"] > 1940]

In [3]:
df_all

,sexe,prenom,periode,niveau_geographique,geographie,valeur
1158801,1,HENRI,1941,DEP,01,45
1158831,1,HENRI,1941,DEP,32,10
1158845,1,HENRI,1941,DEP,46,10
1158802,1,HENRI,1941,DEP,02,25
6391461,2,YOLANDE,1941,DEP,30,15
...,...,...,...,...,...,...
1838832,1,MANOÉ,2024,REG,93,10
5717497,2,NOLA,2024,DEP,44,5
5717498,2,NOLA,2024,DEP,69,5
1838825,1,MANOÉ,2024,REG,32,40


In [4]:
df = df_all[df_all["niveau_geographique"] == "FRANCE"]
df = df.drop(columns=["niveau_geographique", "geographie"])

In [5]:
df["prenom"] = df["prenom"].apply(lambda s: s.lower())
df = df.rename(columns = {"periode": "year", "prenom": "name", "valeur": "count", "sexe": "gender"})

In [6]:
# le chat: aggréger M/F
grouped = df.groupby(['name', 'year', 'gender'])['count'].sum().reset_index()
total_counts = grouped.groupby(['name', 'year'])['count'].transform('sum')
grouped['total_count'] = total_counts
grouped['percentage'] = (grouped['count'] / grouped['total_count'])
result = grouped.pivot(index=['name', 'year'], columns='gender', values='percentage').reset_index()
result = result.merge(grouped[['name', 'year', 'total_count']].drop_duplicates(), on=['name', 'year'])
result = result.fillna(0).rename(columns={"1": "M", "2": "F", "total_count": "count"}).sort_values("year")
df = result
df["count_f"] = df["count"] * df["F"]

In [7]:
df

,name,year,M,F,count,count_f
360910,marcelle,1941,0.0,1.0,1745,1745.0
360985,marcellin,1941,1.0,0.0,15,0.0
27374,alphonso,1941,1.0,0.0,5,0.0
487625,rollande,1941,0.0,1.0,25,25.0
329082,louis,1941,1.0,0.0,3280,0.0
...,...,...,...,...,...,...
446710,nuno,2024,1.0,0.0,35,0.0
605103,ömer-asaf,2024,1.0,0.0,5,0.0
605139,özlem,2024,0.0,1.0,5,5.0
605108,öykü,2024,0.0,1.0,5,5.0


In [8]:
df["pair"] = df.apply(lambda x: [x["year"], x["count"]], axis=1)
df

,name,year,M,F,count,count_f,pair
360910,marcelle,1941,0.0,1.0,1745,1745.0,"[1941, 1745]"
360985,marcellin,1941,1.0,0.0,15,0.0,"[1941, 15]"
27374,alphonso,1941,1.0,0.0,5,0.0,"[1941, 5]"
487625,rollande,1941,0.0,1.0,25,25.0,"[1941, 25]"
329082,louis,1941,1.0,0.0,3280,0.0,"[1941, 3280]"
...,...,...,...,...,...,...,...
446710,nuno,2024,1.0,0.0,35,0.0,"[2024, 35]"
605103,ömer-asaf,2024,1.0,0.0,5,0.0,"[2024, 5]"
605139,özlem,2024,0.0,1.0,5,5.0,"[2024, 5]"
605108,öykü,2024,0.0,1.0,5,5.0,"[2024, 5]"


In [9]:
stats = df.groupby('name').agg(
    count=('count', 'sum'),
    #avg=('count', 'mean'),
    #max=('count', 'max'),
    #min=('count', 'min'),
    #std=('count', 'std'),
    count_f=('count_f', 'sum'),
    years=('pair', list),
).reset_index().fillna(0)
stats["F"] = stats["count_f"] / stats["count"]
stats = stats.drop(columns=['count_f'])
stats["years"] = stats["years"].map(lambda x: json.dumps(x))

last_10_years = df[(df['year'] >= 2016)]
last_10_years_counts = last_10_years.groupby('name')['count'].sum().reset_index()
last_10_years_counts.columns = ['name', 'recent_count']
stats = stats.merge(last_10_years_counts, on='name', how='left')

# Filter out names that aren't common enough
stats = stats[(stats["recent_count"] > 300) | ((stats["count"] > 5_000) & (stats["recent_count"] > 0))]

stats["recent_count_per_k"] = stats["recent_count"] * 1000 / stats["recent_count"].sum()
stats["count_per_k"] = stats["count"] * 1000 / stats["count"].sum()

stats

,name,count,years,F,recent_count,recent_count_per_k,count_per_k
33,aaliyah,3900,"[[1998, 5], [1999, 5], [2000, 5], [2001, 30], ...",1.000000,2315.0,0.451216,0.066234
55,aaron,37965,"[[1949, 5], [1960, 5], [1962, 5], [1963, 5], [...",0.002107,20485.0,3.992730,0.644763
110,abby,3030,"[[1990, 5], [1997, 5], [2002, 5], [2003, 5], [...",1.000000,1880.0,0.366431,0.051459
120,abd,1680,"[[1947, 5], [1949, 5], [1950, 5], [1951, 10], ...",0.000000,315.0,0.061397,0.028532
142,abdallah,6575,"[[1944, 5], [1947, 5], [1948, 5], [1949, 10], ...",0.000000,1780.0,0.346940,0.111664
...,...,...,...,...,...,...,...
46964,éva,2930,"[[1972, 5], [1973, 5], [1979, 5], [1980, 5], [...",1.000000,1905.0,0.371303,0.049760
46965,évan,1150,"[[2010, 30], [2011, 75], [2012, 80], [2013, 90...",0.000000,730.0,0.142284,0.019531
46993,ézio,720,"[[2011, 15], [2012, 15], [2013, 20], [2014, 20...",0.000000,625.0,0.121819,0.012228
46997,ézéchiel,495,"[[2010, 5], [2011, 5], [2012, 5], [2013, 20], ...",0.000000,425.0,0.082837,0.008407


In [10]:
# The batches are to be forwarded to an LLM
# While it can (and likely does) fail every now and then,
# it's surprisingly reliable. 
phonetic = pd.read_csv("phonetic.csv")
missing = list(set(stats.name.values) - set(phonetic.name.values))
batch_size = 600 # More than that and the LLM usually stops before the end
for i in range(10):
    batch = missing[(i * batch_size):((i + 1) * batch_size)]
    if not batch:
        break
    print(batch)

In [11]:
phonetic = pd.read_csv("phonetic.csv")
stats = stats.merge(phonetic, on='name', how='left')
stats

,name,count,years,F,recent_count,recent_count_per_k,count_per_k,phonetic
0,aaliyah,3900,"[[1998, 5], [1999, 5], [2000, 5], [2001, 30], ...",1.000000,2315.0,0.451216,0.066234,/a.li.ja/
1,aaron,37965,"[[1949, 5], [1960, 5], [1962, 5], [1963, 5], [...",0.002107,20485.0,3.992730,0.644763,/a.ʁɔn/
2,abby,3030,"[[1990, 5], [1997, 5], [2002, 5], [2003, 5], [...",1.000000,1880.0,0.366431,0.051459,/a.bi/
3,abd,1680,"[[1947, 5], [1949, 5], [1950, 5], [1951, 10], ...",0.000000,315.0,0.061397,0.028532,/abd/
4,abdallah,6575,"[[1944, 5], [1947, 5], [1948, 5], [1949, 10], ...",0.000000,1780.0,0.346940,0.111664,/ab.da.la/
...,...,...,...,...,...,...,...,...
2668,éva,2930,"[[1972, 5], [1973, 5], [1979, 5], [1980, 5], [...",1.000000,1905.0,0.371303,0.049760,/e.va/
2669,évan,1150,"[[2010, 30], [2011, 75], [2012, 80], [2013, 90...",0.000000,730.0,0.142284,0.019531,/e.vɑ̃/
2670,ézio,720,"[[2011, 15], [2012, 15], [2013, 20], [2014, 20...",0.000000,625.0,0.121819,0.012228,/e.zjo/
2671,ézéchiel,495,"[[2010, 5], [2011, 5], [2012, 5], [2013, 20], ...",0.000000,425.0,0.082837,0.008407,/e.ze.kjɛl/


In [12]:
relative_f = (stats['F'] * stats['count']).groupby(stats['phonetic']).sum() / stats.groupby('phonetic')['count'].sum()
spellings = stats.groupby(['phonetic', 'name'])['count'].sum().reset_index()
spellings['relative_frequency'] = spellings['count'] / spellings.groupby('phonetic')['count'].transform('sum')
spellings_dict = spellings.groupby('phonetic').apply(
    lambda x: json.dumps(x[['name', 'relative_frequency']].to_dict('records')), include_groups=False
).to_dict()

result_df = pd.DataFrame({
    'phonetic': relative_f.index,
    'phonetic_relative_f': relative_f.values
})
result_df['other_spellings'] = result_df['phonetic'].map(spellings_dict)
result_df

,phonetic,phonetic_relative_f,other_spellings
0,/a.bi.ɡa.il/,0.994083,"[{""name"": ""abiga\u00efl"", ""relative_frequency""..."
1,/a.bi.ɡa.ɛl/,0.996424,"[{""name"": ""abiga\u00ebl"", ""relative_frequency""..."
2,/a.bi.ɡɛl/,1.000000,"[{""name"": ""abigail"", ""relative_frequency"": 1.0}]"
3,/a.bi/,1.000000,"[{""name"": ""abby"", ""relative_frequency"": 0.5564..."
4,/a.bib/,0.000000,"[{""name"": ""habib"", ""relative_frequency"": 1.0}]"
...,...,...,...
1991,/ʒɛ.ni/,0.997906,"[{""name"": ""jenny"", ""relative_frequency"": 1.0}]"
1992,/ʒɛ.si/,0.906939,"[{""name"": ""jessie"", ""relative_frequency"": 1.0}]"
1993,/ʒɛ.zɔ̃/,0.000000,"[{""name"": ""jason"", ""relative_frequency"": 1.0}]"
1994,/ʒɛʁ.mɛn/,0.999481,"[{""name"": ""germaine"", ""relative_frequency"": 1.0}]"


In [13]:
with_phonetics = pd.merge(stats, result_df, on="phonetic").set_index("name")
with_phonetics.sample(5)

,count,years,F,recent_count,recent_count_per_k,count_per_k,phonetic,phonetic_relative_f,other_spellings
name,,,,,,,,,
ugo,11020,"[[1943, 5], [1951, 5], [1953, 5], [1954, 5], [...",0.000000,1180.0,0.229994,0.187154,/y.go/,0.000000,"[{""name"": ""ugo"", ""relative_frequency"": 1.0}]"
christiane,163725,"[[1941, 6255], [1942, 7325], [1943, 8005], [19...",1.000000,40.0,0.007796,2.780558,/kʁis.tjan/,1.000000,"[{""name"": ""christiane"", ""relative_frequency"": ..."
camélia,9705,"[[1941, 5], [1942, 5], [1943, 5], [1944, 5], [...",1.000000,3305.0,0.644177,0.164821,/ka.me.lja/,1.000000,"[{""name"": ""camelia"", ""relative_frequency"": 0.0..."
maëlia,1765,"[[2010, 35], [2011, 90], [2012, 115], [2013, 1...",1.000000,1065.0,0.207579,0.029975,/ma.e.lja/,1.000000,"[{""name"": ""ma\u00e9lia"", ""relative_frequency"":..."
yolande,31100,"[[1941, 700], [1942, 855], [1943, 935], [1944,...",0.999839,15.0,0.002924,0.528174,/jɔ.lɑ̃d/,0.999839,"[{""name"": ""yolande"", ""relative_frequency"": 1.0}]"


In [14]:
with_phonetics.to_csv("name_stats.csv")